In [1]:
import os
import json


In [2]:
# Open and read the config file
with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

# Retrieve the API key from the config data
api_key = config_data['api_key']
os.environ['OPENAI_API_KEY'] = api_key

## Load the index

In [3]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_storage(storage_context)

In [9]:
chat_engine = index.as_chat_engine()
response = chat_engine.chat("what is LiquidEarth?")
print(response)


LiquidEarth is a cloud-based solution for the visualization, processing, and analysis of 3D geological data, models, and more. It is relevant to the conversation we were having because it provides a way to visualize, process, and analyze 3D geological data, models, and more.


## GPT 4

In [6]:
from llama_index.llms import OpenAI
from llama_index import ServiceContext

gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

### create Prompt Template

In [18]:
from llama_index import Prompt
# define custom Prompt
TEMPLATE_STR = (
    "You are Leia, the LiquidEarth Intelligent Assistant. You are helping a user with a question about LiquidEarth. You are very smart and friendly and always in a great mood.\n"
    "In LiquidEarth, a 'Space' and a 'Project are the same thing. We have provided Documentation on the software and further information below. In some cases the metadata includes a 'Control' Field that points to a UI element in the app associated to the described functionality. this is only for internal use. when describing controls to the user, use descriptions and names from the text, not the 'control' values. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Answer the question for a human to understand. Additionally, return the 'Control' properties in the order of operations in the following form at the end of your response: [[control1], [control2], ...]. Append the list to your response without further comment. If no controls are found, do not comment it. Never include any controls that are not specified in the Metadata Field in the provided documentation. Do not interpret any controls from the text body. If the answer requires multiple steps, describe each step in detail. Given this information, please answer the question: {query_str}\n"
)
QA_TEMPLATE = Prompt(TEMPLATE_STR)

In [19]:
query_engine = index.as_query_engine(service_context=service_context_gpt4, text_qa_template=QA_TEMPLATE, retriever_mode="embedding")

In [20]:
#dirty hack: trying to increase the context size
query_engine.retriever._similarity_top_k = 6

In [25]:
response = query_engine.query("hello,how can i create a project and add some data?")

In [26]:
print(response)

Hello! Here's how you can create a project and add data in LiquidEarth:

**Creating a New Project:**

1. Open the server explorer on the left side of your screen.
2. Click the Plus button at the bottom of your server explorer list to create a new project.
3. If you want to modify basic information and project metadata (like project name and owner), use the Edit Data option in the inspector.
    - After setting all required fields, click Save to confirm the information.
4. Use the Load Project option in the inspector to load the project into your local workspace (3D Workspace). The project will appear in your 3D workspace and will be listed in the Local Explorer.
    - A message will appear in the activity log indicating the loading progress. If you're working online, the project will also be automatically uploaded to the cloud or the remote server you're connected to.

The newly created project space in your 3D Workspace will be visualized by a bounding box that covers the extent defin

In [17]:
response


Response(response="I'm an artificial intelligence and don't have feelings, but I'm functioning as expected! Thank you for asking.", source_nodes=[NodeWithScore(node=TextNode(id_='254ceab8-47f1-4c33-b0e1-3cb2f3f725bd', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='71e204eb-d60b-4374-9cce-85886297219b', node_type=None, metadata={}, hash='e1cfbde5e49b0b7d9d1ace99337db6ce97a89788792f23c70c8fcdd1f6bd0ad7')}, hash='62c36937451b513e0c891164fbbea41eb7854312a1b21dc37d8fa74b18e580ae', text='Basics\n\n---', start_char_idx=0, end_char_idx=11, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.781250216315416), NodeWithScore(node=TextNode(id_='e2f95d75-00d7-4048-9846-42716d975f78', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>:

In [15]:
response

Response(response='The context information provided does not include any specific control properties from the documentation.', source_nodes=[NodeWithScore(node=TextNode(id_='231e7e6b-7b24-48f2-84cf-2cdd81c626e2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='23c7fda5-89da-4807-836a-338ef9e4d60b', node_type=None, metadata={}, hash='4ac1dbe521652d933f25b47af71ba8445e3925649cd40bf4843095ce65dcf733')}, hash='b161860badf44541bc86a3ed070fab56fecc17b2e3963915456dcfa212931116', text='Other Resources\n\n---\n\nAll Documentation Entries', start_char_idx=0, end_char_idx=47, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7849898990786808), NodeWithScore(node=TextNode(id_='bae2ddf0-57da-42bd-9faa-d9c5a02f0cc8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationship

## Chat with a prompt template (ToDo)

In [ ]:
custom_prompt = Prompt("""\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
""")

# list of (human_message, ai_message) tuples
custom_chat_history = [
    (
        'Hello assistant, we are having a insightful discussion about Paul Graham today.',
        'Okay, sounds good.'
    )
]

query_engine = index.as_query_engine()
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    verbose=True
)